# Software Coaching for Python
# Week 5: Document Clustering & Topic Modeling - Homework

Instructor: Kang-Pyo Lee

In [2]:
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/Shared drives/PythonCoaching_2023Summer"
my_folder = "ljw33083418"     # *** REPLACE WITH YOUR FOLDER NAME ***
outcome_folder = f"{path}/{my_folder}/outcome"
classdata_folder = f"{path}/classdata"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install --upgrade scikit-learn pyLDAvis==3.4.1

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

df = pd.read_csv(f"{classdata_folder}/timeline_cnnbrk.csv", sep="\t")
df.text = df.text.astype(str)
df

,status_id,created_at,text,retweet_count,favorite_count,is_retweet,retweet_created_at
0,1312243714585235457,Sat Oct 03 04:10:56 +0000 2020,"President Donald Trump's campaign manager Bill Stepien has tested positive for coronavirus, according to a senior o… https://t.co/piwok6WXpA",1050,3946,0,NaN
1,1312217447752237056,Sat Oct 03 02:26:34 +0000 2020,Former White House adviser Kellyanne Conway says she has tested positive for Covid-19. She attended a Rose Garden e… https://t.co/DzHyaaO7Cj,2287,7232,0,NaN
2,1312189828247691265,Sat Oct 03 00:36:49 +0000 2020,Sen. Thom Tillis says he has tested positive for Covid-19. He's the second member of the Senate Judiciary Committee… https://t.co/J3cnZUTEjp,501,1559,0,NaN
3,1312188613015203842,Sat Oct 03 00:31:59 +0000 2020,"After Joe Biden's campaign said it would pull negative ads following President Trump's Covid-19 diagnosis, the Pres… https://t.co/hArgCeKRzC",2831,5992,0,NaN
4,1312161028923244545,Fri Oct 02 22:42:22 +0000 2020,"President Trump has landed on Marine One at Walter Reed medical center, where he will be hospitalized for the next… https://t.co/Btb0FBkJLW",249,841,0,NaN
...,...,...,...,...,...,...,...
3195,1231426815996170243,Sun Feb 23 03:53:26 +0000 2020,Science Channel star Mad Mike Hughes died Saturday during an attempt to launch his homemade rocket… https://t.co/ezI1BUHstT,345,961,0,NaN
3196,1231402873617108992,Sun Feb 23 02:18:18 +0000 2020,"A person was killed in New Orleans when they were hit by a float during the Krewe of Endymion parade, the city says… https://t.co/wC9pSiSqTv",194,345,0,NaN
3197,1231385843723636739,Sun Feb 23 01:10:37 +0000 2020,"Sen. Bernie Sanders will win a decisive victory in Nevada's Democratic caucuses, CNN projects, solidifying his posi… https://t.co/Wk2nFjEiOw",321,2058,0,NaN
3198,1231338606863601668,Sat Feb 22 22:02:55 +0000 2020,"Three people were killed and 18 injured after a bus rolled over on Interstate 15 near Fallbrook, California, north… https://t.co/UHqTVpV7Vm",97,305,0,NaN


Check if there are any duplicates in the `text` column.

In [5]:
df.text.value_counts()

text
President Donald Trump's campaign manager Bill Stepien has tested positive for coronavirus, according to a senior o… https://t.co/piwok6WXpA    1
Cook County, Illinois, has replaced Queens County, New York, as the single US county with the highest number of cor… https://t.co/fP1HIZUnZx    1
The CDC director says the department's models predict more than 100,000 US Covid-19 deaths by June 1… https://t.co/3PECBOa3mG                   1
Kroger, the largest US supermarket chain, is ending its $2 per hour hero pay for employees and providing a one-ti… https://t.co/8USEZwwXZQ      1
A US F-22 stealth fighter jet crashed Friday during a routine training flight in Florida, the Air Force said. The… https://t.co/yDsi6fNzdO      1
                                                                                                                                               ..
Miami Beach announces an 8 p.m. curfew for most of its entertainment district, beginning Saturday… https://t.co/mplbvqV

Remove duplicates in the `text` column, if any.

In [6]:
df = df.drop_duplicates(["text"], keep="first").copy()

In [7]:
df.text.value_counts()

text
President Donald Trump's campaign manager Bill Stepien has tested positive for coronavirus, according to a senior o… https://t.co/piwok6WXpA    1
Cook County, Illinois, has replaced Queens County, New York, as the single US county with the highest number of cor… https://t.co/fP1HIZUnZx    1
The CDC director says the department's models predict more than 100,000 US Covid-19 deaths by June 1… https://t.co/3PECBOa3mG                   1
Kroger, the largest US supermarket chain, is ending its $2 per hour hero pay for employees and providing a one-ti… https://t.co/8USEZwwXZQ      1
A US F-22 stealth fighter jet crashed Friday during a routine training flight in Florida, the Air Force said. The… https://t.co/yDsi6fNzdO      1
                                                                                                                                               ..
Miami Beach announces an 8 p.m. curfew for most of its entertainment district, beginning Saturday… https://t.co/mplbvqV

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   status_id           3200 non-null   int64 
 1   created_at          3200 non-null   object
 2   text                3200 non-null   object
 3   retweet_count       3200 non-null   int64 
 4   favorite_count      3200 non-null   int64 
 5   is_retweet          3200 non-null   int64 
 6   retweet_created_at  348 non-null    object
dtypes: int64(4), object(3)
memory usage: 175.1+ KB


## Preparing Data for Modeling

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])

global_stopwords = stopwords.words("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Define your own local stopwords.

In [10]:
import string

local_stopwords = [c for c in string.punctuation] +\
                  ['‘', '’', '—', '…', '“', '”', "'re", "'s", '...', 'a…', 't…', 'https', "n't", 'rt']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm="l2", stop_words=global_stopwords+local_stopwords, max_df=0.7)
X = vectorizer.fit_transform(df.text)

In [12]:
X.shape

(3200, 10866)

## K-Means Clustering

### Step 1. Choose the number of clusters

Set the number of clusters you would like to get.

In [13]:
# Your answer here
k = 5

### Step 2. Initialize a model object for k-means clustering

In [14]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, random_state=0)
kmeans

KMeans(n_clusters=5, random_state=0)

### Step 3. Fit the model using the input data

In [15]:
%time kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


CPU times: user 263 ms, sys: 8.2 ms, total: 271 ms
Wall time: 295 ms


KMeans(n_clusters=5, random_state=0)

### Step 4. Examine the clustering outcome

In [16]:
kmeans.labels_

array([1, 3, 3, ..., 2, 0, 0], dtype=int32)

In [17]:
df["cluster"] = kmeans.labels_

In [18]:
df[["text", "cluster"]]

,text,cluster
0,"President Donald Trump's campaign manager Bill Stepien has tested positive for coronavirus, according to a senior o… https://t.co/piwok6WXpA",1
1,Former White House adviser Kellyanne Conway says she has tested positive for Covid-19. She attended a Rose Garden e… https://t.co/DzHyaaO7Cj,3
2,Sen. Thom Tillis says he has tested positive for Covid-19. He's the second member of the Senate Judiciary Committee… https://t.co/J3cnZUTEjp,3
3,"After Joe Biden's campaign said it would pull negative ads following President Trump's Covid-19 diagnosis, the Pres… https://t.co/hArgCeKRzC",3
4,"President Trump has landed on Marine One at Walter Reed medical center, where he will be hospitalized for the next… https://t.co/Btb0FBkJLW",1
...,...,...
3195,Science Channel star Mad Mike Hughes died Saturday during an attempt to launch his homemade rocket… https://t.co/ezI1BUHstT,0
3196,"A person was killed in New Orleans when they were hit by a float during the Krewe of Endymion parade, the city says… https://t.co/wC9pSiSqTv",4
3197,"Sen. Bernie Sanders will win a decisive victory in Nevada's Democratic caucuses, CNN projects, solidifying his posi… https://t.co/Wk2nFjEiOw",2
3198,"Three people were killed and 18 injured after a bus rolled over on Interstate 15 near Fallbrook, California, north… https://t.co/UHqTVpV7Vm",0


Check the number of values for each cluster.

In [19]:
df.cluster.value_counts()

cluster
0    1886
1     809
4     205
3     193
2     107
Name: count, dtype: int64

Check 10 random texts from the largest cluster.

In [21]:
#df[df.cluster == 1].sample(10, random_state=0)[["text", "cluster"]]
df[df.cluster == 0].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
366,Eight defendants in journalist Jamal Khashoggi's murder have been sentenced in Saudi Arabia. A UN official calls th… https://t.co/PnlWgBuy8a,0
810,"E! News, the entertainment news show launched in 1991, has been canceled in response to the economic fallout from t… https://t.co/3Brn8Rs4Lx",0
2926,The United Kingdom Department of Health and Social Care confirms a total of 21 people have died after contracting c… https://t.co/sjbecdkwDw,0
2706,"RT @CNNBusiness: US stocks finished higher, recording their third straight day of gains despite the worst jobless claims data in history. T…",0
148,The Metropolitan Opera announces it has canceled the entire 2020-21 season due to the ongoing health crisis… https://t.co/ALWjLBdkKY,0
73,RT @CNNPolitics: Census Bureau says operations will conclude by October 5 https://t.co/3ajTKpyd7Y https://t.co/88WwSTWijj,0
1496,"The CDC will make an updated recommendation on masks soon, a senior agency official says https://t.co/97WPKPVHcu https://t.co/AKdVVzEhRK",0
1889,Seattle's mayor has announced a citywide curfew from 5 p.m. to 5 a.m. for both Saturday and Sunday in wake of viole… https://t.co/xJ6ZfG0iUp,0
612,"US intel shows that officials in Wuhan, China, kept senior officials in Beijing in the dark for weeks about coronav… https://t.co/8CWYi9j5Ss",0
2052,A Senate panel votes to issue a subpoena as part of the GOP-led investigation into the Ukrainian energy firm that h… https://t.co/cd9FWR6OuW,0


Check 10 random texts from the second largest cluster.

In [22]:
#df[df.cluster == 2].sample(10, random_state=0)[["text", "cluster"]]
df[df.cluster == 1].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
2714,RT @CNNPolitics: BREAKING: Senate approves historic $2 trillion stimulus deal amid growing coronavirus fears https://t.co/1mxJCqpSUe https:…,1
1007,Dr. Anthony Fauci says he's not eating at restaurants right now and can't think of a reason to get on a transatla… https://t.co/hIBgjOZmRm,1
219,Senate Majority Leader McConnell says President Trump's nominee to replace Justice Ginsburg will receive a vote on… https://t.co/5fEKn0ullp,1
2352,RT @CNNPolitics: President Trump erupts at campaign manager as reelection stress overflows https://t.co/Kqht1KYSPV https://t.co/Z6hMgBF1nj,1
88,"House Speaker Nancy Pelosi criticizes Trump's swift move to make a Supreme Court pick, saying he's moving quickly t… https://t.co/QIWxh3XI1q",1
39,"Brad Parscale has stepped down from his role as a senior adviser to President Trump's campaign, two sources close t… https://t.co/ar7tcoawPX",1
2270,Tyson Foods says its largest US pork plant will reopen Thursday after a major coronavirus outbreak forced the compa… https://t.co/fo5rmxjprV,1
2389,Children in Spain are allowed out of the house for the first time in six weeks as coronavirus deaths in the country… https://t.co/j8gaOqVXrU,1
1210,"The Supreme Court says Trump is not immune from subpoena, but NY prosecutor won't get financial papers yet. Ruling… https://t.co/0V7YN8ojw1",1
3049,"Colorado officials announced a presumptive positive case of coronavirus, the state’s first case. Follow live update… https://t.co/SYe61nW5zt",1


In [23]:
import nltk
df["words"] = df.text.apply(lambda x: nltk.word_tokenize(x))
df["tagged_words"] = df.words.apply(lambda x: nltk.pos_tag(x))

from collections import Counter

def get_counter(dataframe, stopwords=[]):
    counter = Counter()

    for l in dataframe.tagged_words:
        word_set = set()

        for t in l:
            word = t[0].lower()
            tag = t[1]

            if word not in stopwords:
                word_set.add(word)

        counter.update(word_set)

    return counter

Check the top-30 most common keywords in the largest cluster. (Replace the cluster number as needed.)

In [24]:
counter0 = get_counter(df[df.cluster == 1], global_stopwords+local_stopwords)
counter0.most_common(30)

[('coronavirus', 428),
 ('trump', 340),
 ('president', 263),
 ('says', 165),
 ('us', 150),
 ('cnnpolitics', 114),
 ('house', 70),
 ('positive', 57),
 ('white', 55),
 ('tested', 54),
 ('cases', 50),
 ('people', 49),
 ('first', 48),
 ('administration', 48),
 ('new', 40),
 ('pandemic', 39),
 ('due', 39),
 ('according', 35),
 ('health', 34),
 ('former', 33),
 ('court', 30),
 ('dr.', 30),
 ('federal', 29),
 ('least', 29),
 ('two', 27),
 ('university', 27),
 ('died', 27),
 ('cnn', 26),
 ('national', 26),
 ('state', 26)]

Check the top-30 most common keywords in the second largest cluster. (Replace the cluster number as needed.)

In [25]:
counter4 = get_counter(df[df.cluster == 2], global_stopwords+local_stopwords)
counter4.most_common(30)

[('biden', 95),
 ('joe', 94),
 ('cnnpolitics', 61),
 ('democratic', 44),
 ('president', 40),
 ('cnn', 34),
 ('former', 31),
 ('primary', 27),
 ('vice', 26),
 ('projects', 26),
 ('win', 26),
 ('breaking', 22),
 ('cnnelection', 19),
 ('bernie', 17),
 ('sanders', 17),
 ('sen.', 16),
 ('says', 14),
 ('trump', 13),
 ('presidential', 12),
 ('running', 8),
 ('nominee', 7),
 ('nomination', 7),
 ('mate', 7),
 ('endorse', 7),
 ('endorses', 7),
 ('follow', 6),
 ('amy', 6),
 ('woman', 6),
 ('campaign', 6),
 ('coronavirus', 5)]

## LDA Topic Modeling

### Step 1. Choose the number of topics

Set the number of topics you would like to get.

In [26]:
# Your answer here
num_topics = 5

### Step 2. Initialize a model object for LDA topic modeling

In [27]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=num_topics, random_state=0)
lda

LatentDirichletAllocation(n_components=5, random_state=0)

### Step 3. Fit the model using the input data

In [28]:
%time lda.fit(X)

CPU times: user 3.92 s, sys: 20.6 ms, total: 3.94 s
Wall time: 3.95 s


LatentDirichletAllocation(n_components=5, random_state=0)

### Step 4. Examine the output of topic modeling

In [29]:
def show_topics(model, feature_names, num_top_words):
    for topic_idx, topic_scores in enumerate(model.components_):
        print("***Topic {}:".format(topic_idx))
        print(" + ".join(["{:.2f} * {}".format(topic_scores[i], feature_names[i]) for i in topic_scores.argsort()[::-1][:num_top_words]]))
        print()

In [30]:
show_topics(lda, vectorizer.get_feature_names_out(), 10)

***Topic 0:
16.94 * says + 14.99 * cnnpolitics + 13.58 * president + 13.45 * us + 12.08 * 19 + 11.83 * covid + 11.56 * trump + 10.62 * dr + 10.11 * democratic + 10.01 * hurricane

***Topic 1:
18.07 * coronavirus + 11.99 * us + 11.66 * new + 11.38 * says + 9.69 * police + 9.14 * trump + 7.88 * president + 7.62 * cases + 7.30 * 000 + 7.23 * people

***Topic 2:
14.07 * says + 12.04 * trump + 11.23 * coronavirus + 11.02 * us + 10.28 * new + 10.17 * president + 9.29 * house + 8.48 * cnnpolitics + 7.32 * state + 6.52 * york

***Topic 3:
14.37 * new + 12.83 * coronavirus + 9.14 * says + 9.10 * york + 8.87 * us + 8.50 * trump + 8.05 * people + 7.74 * gov + 7.48 * president + 6.99 * state

***Topic 4:
19.13 * coronavirus + 17.00 * us + 16.30 * trump + 13.31 * president + 13.09 * says + 11.11 * people + 10.84 * cases + 8.32 * least + 8.06 * new + 8.02 * 000



### Topic Model Visualization

In [31]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()

In [32]:
pyLDAvis.lda_model.prepare(lda, X, vectorizer)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.057965 -0.004400       1        1  22.258643
4     -0.016480 -0.043067       2        1  21.118703
2     -0.005147  0.028663       3        1  19.076250
1     -0.019515 -0.003379       4        1  18.815662
3     -0.016822  0.022182       5        1  18.730742, topic_info=            Term      Freq      Total Category  logprob  loglift
4764   hurricane  6.000000   6.000000  Default  30.0000  30.0000
3163          dr  7.000000   7.000000  Default  29.0000  29.0000
3774       fauci  4.000000   4.000000  Default  28.0000  28.0000
10363        win  5.000000   5.000000  Default  27.0000  27.0000
9058       storm  4.000000   4.000000  Default  26.0000  26.0000
...          ...       ...        ...      ...      ...      ...
5287         joe  2.234926  11.309699   Topic5  -6.8357   0.0536
3912       first  2.300567  13.899723   Topic5  -6.8068  -0.1237
5715       least  2.185806  10.925918   Topic5  -6.8579   0.0659
9715         two  2.157142  13.210978   Topic5  -6.8711  -0.1373
7333      police  2.116107  16.732621   Topic5  -6.8904  -0.3928

[384 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
1          1  0.147912   000
1          2  0.295823   000
1          3  0.147912   000
1          4  0.295823   000
1          5  0.073956   000
...      ...       ...   ...
10723      1  0.133230  york
10723      2  0.199846  york
10723      3  0.199846  york
10723      4  0.133230  york
10723      5  0.266461  york

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 2, 4])